In [1]:
#setup and install
!pip -q install torchmetrics grad-cam timm==1.0.9
!pip -q install kaggle

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 66.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 89.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.0/983.0 kB 59.7 MB/s eta 0:00:00


In [2]:
#mount google drive
from google.colab import drive
drive.mount('/content/drive')  # click through the auth
SAVE_DIR = "/content/drive/MyDrive/ham10000_models"  # change if you want
import os; os.makedirs(SAVE_DIR, exist_ok=True)


Mounted at /content/drive


In [7]:
from google.colab import files
uploaded = files.upload()   # 👉 Upload the kaggle.json you got from Kaggle account


Saving kaggle (1).json to kaggle (1) (2).json


In [8]:
!mkdir -p /root/.kaggle
!mv kaggle*.json /root/.kaggle/kaggle.json   # this will handle "kaggle (1).json" etc.
!chmod 600 /root/.kaggle/kaggle.json


mv: target '/root/.kaggle/kaggle.json' is not a directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory


In [12]:
# --- STEP 2 (Corrected): Get HAM10000 from Kaggle & arrange files ---

import os, glob, shutil, zipfile

DATA_ROOT = "/content/HAM10000"
IMAGES_DIR = f"{DATA_ROOT}/images"
META_CSV   = f"{DATA_ROOT}/HAM10000_metadata.csv"

os.makedirs(DATA_ROOT, exist_ok=True)
os.makedirs(IMAGES_DIR, exist_ok=True)

def dataset_ready():
    return os.path.exists(META_CSV) and len(glob.glob(f"{IMAGES_DIR}/*.jpg")) >= 10000

# 1) If dataset already complete, skip download
if dataset_ready():
    print("✅ Dataset already ready at", DATA_ROOT)
else:
    # 2) Ensure Kaggle API token exists at /root/.kaggle/kaggle.json
    os.makedirs("/root/.kaggle", exist_ok=True)
    kaggle_token_path = "/root/.kaggle/kaggle.json"
    if not os.path.exists(kaggle_token_path):
        # Try to find any uploaded kaggle*.json (handles names like 'kaggle (1).json')
        cand = glob.glob("/content/kaggle*.json")
        if not cand:
            # Prompt upload if not found
            from google.colab import files
            print("📥 Please upload your Kaggle API token (kaggle.json).")
            uploaded = files.upload()  # pick your kaggle.json file
            # After upload, it will live in /content
            cand = glob.glob("/content/kaggle*.json")
            if not cand:
                raise FileNotFoundError("kaggle.json not uploaded. Re-run this cell and upload the file when prompted.")
        # Move first candidate to the expected path
        src = sorted(cand)[0]
        shutil.move(src, kaggle_token_path)
        print(f"🔑 Placed Kaggle token at {kaggle_token_path}")
    # Set permissions
    !chmod 600 /root/.kaggle/kaggle.json

    # 3) Download (or re-download) the dataset zip to /content
    zip_path = "/content/skin-cancer-mnist-ham10000.zip"
    if not os.path.exists(zip_path):
        print("⬇️  Downloading HAM10000 from Kaggle...")
        !kaggle datasets download -d kmader/skin-cancer-mnist-ham10000 -p /content
    else:
        print("ℹ️  Found existing zip at /content; reusing it.")

    # 4) Extract to /content (quietly)
    if not os.path.exists("/content/HAM10000_metadata.csv") or not glob.glob("/content/HAM10000_images_part_1/*.jpg"):
        print("🗜️  Extracting zip...")
        !unzip -qo /content/skin-cancer-mnist-ham10000.zip -d /content
    else:
        print("ℹ️  Found extracted contents; reusing them.")

    # 5) Move images into one folder and move metadata CSV
    #    (idempotent: only move files that aren't already in destination)
    moved_imgs = 0
    for part in ["HAM10000_images_part_1","HAM10000_images_part_2"]:
        src_dir = f"/content/{part}"
        if os.path.isdir(src_dir):
            for p in glob.glob(f"{src_dir}/*.jpg"):
                dest = f"{IMAGES_DIR}/{os.path.basename(p)}"
                if not os.path.exists(dest):
                    shutil.move(p, dest)
                    moved_imgs += 1
    # Move metadata CSV
    if os.path.exists("/content/HAM10000_metadata.csv") and not os.path.exists(META_CSV):
        shutil.move("/content/HAM10000_metadata.csv", META_CSV)

    # 6) Final sanity checks
    n_imgs = len(glob.glob(f"{IMAGES_DIR}/*.jpg"))
    ok = os.path.exists(META_CSV) and n_imgs >= 10000
    print(f"📦 Images in {IMAGES_DIR}: {n_imgs}")
    print("📄 Metadata CSV:", "present" if os.path.exists(META_CSV) else "missing")
    if not ok:
        raise RuntimeError(
            "Dataset setup incomplete. Check that your Kaggle download succeeded and rerun this cell."
        )
    print("✅ Dataset prepared at:", DATA_ROOT)


🔑 Placed Kaggle token at /root/.kaggle/kaggle.json
⬇️  Downloading HAM10000 from Kaggle...
Dataset URL: https://www.kaggle.com/datasets/kmader/skin-cancer-mnist-ham10000
License(s): CC-BY-NC-SA-4.0
 99% 5.13G/5.20G [01:22<00:04, 15.4MB/s]
100% 5.20G/5.20G [01:22<00:00, 67.3MB/s]
🗜️  Extracting zip...
📦 Images in /content/HAM10000/images: 10015
📄 Metadata CSV: present
✅ Dataset prepared at: /content/HAM10000


In [13]:
# --- STEP 3: Imports, reproducibility, and load metadata ---

import os, random
import numpy as np
import pandas as pd
import torch

# Reproducibility
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print("Device:", DEVICE)

DATA_ROOT  = "/content/HAM10000"
IMAGES_DIR = f"{DATA_ROOT}/images"
META_CSV   = f"{DATA_ROOT}/HAM10000_metadata.csv"

# Sanity checks (Step 2 should have created these)
assert os.path.exists(META_CSV), f"❌ Metadata not found at {META_CSV}. Please re-run Step 2."
assert os.path.isdir(IMAGES_DIR), f"❌ Images folder not found at {IMAGES_DIR}. Please re-run Step 2."

# Load metadata
meta = pd.read_csv(META_CSV)

# Standardize label column name (HAM10000 uses 'dx')
if "dx" in meta.columns and "label" not in meta.columns:
    meta = meta.rename(columns={"dx": "label"})

# Build absolute image paths if missing
if "image_path" not in meta.columns:
    meta["image_path"] = meta["image_id"].apply(lambda x: f"{IMAGES_DIR}/{x}.jpg")

# Keep only rows where image file exists
exists_mask = meta["image_path"].apply(os.path.exists)
missing = (~exists_mask).sum()
if missing > 0:
    print(f"⚠️  {missing} images listed in metadata not found on disk. Dropping those rows.")
meta = meta[exists_mask].reset_index(drop=True)

# Class list & mappings
class_names = sorted(meta["label"].unique().tolist())
cls_to_idx = {c: i for i, c in enumerate(class_names)}
idx_to_cls = {i: c for c, i in cls_to_idx.items()}

print("✅ Metadata loaded.")
print("Classes:", class_names)
print("\nPer-class counts:\n", meta["label"].value_counts().reindex(class_names).to_string())

# Quick preview
print("\nMetadata preview:")
print(meta.head(3))


Device: cuda
✅ Metadata loaded.
Classes: ['akiec', 'bcc', 'bkl', 'df', 'mel', 'nv', 'vasc']

Per-class counts:
 label
akiec     327
bcc       514
bkl      1099
df        115
mel      1113
nv       6705
vasc      142

Metadata preview:
     lesion_id      image_id label dx_type   age   sex localization  \
0  HAM_0000118  ISIC_0027419   bkl   histo  80.0  male        scalp   
1  HAM_0000118  ISIC_0025030   bkl   histo  80.0  male        scalp   
2  HAM_0002730  ISIC_0026769   bkl   histo  80.0  male        scalp   

                                  image_path  
0  /content/HAM10000/images/ISIC_0027419.jpg  
1  /content/HAM10000/images/ISIC_0025030.jpg  
2  /content/HAM10000/images/ISIC_0026769.jpg  


In [18]:
# --- STEP 4 (re-run safely): Create stratified Train/Val/Test splits ---

import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedShuffleSplit

# Sanity: require Step 3 variables
assert 'meta' in globals(), "meta not found. Please run Step 3 first."
assert 'SEED' in globals(), "SEED not found. Please run Step 3 first."
assert 'class_names' in globals(), "class_names not found. Please run Step 3 first."

def stratified_split(df: pd.DataFrame, label_col: str = "label",
                     test_size: float = 0.15, val_size: float = 0.15, seed: int = 42):
    y = df[label_col].values
    idx_all = np.arange(len(df))

    # 1) Hold out test
    sss1 = StratifiedShuffleSplit(n_splits=1, test_size=test_size, random_state=seed)
    train_val_idx, test_idx = next(sss1.split(idx_all, y))
    df_train_val = df.iloc[train_val_idx].reset_index(drop=True)
    df_test      = df.iloc[test_idx].reset_index(drop=True)

    # 2) From remaining, split train vs val
    sss2 = StratifiedShuffleSplit(
        n_splits=1,
        test_size=val_size / (1 - test_size),
        random_state=seed
    )
    idx_tv = np.arange(len(df_train_val))
    train_idx, val_idx = next(sss2.split(idx_tv, df_train_val[label_col].values))
    df_train = df_train_val.iloc[train_idx].reset_index(drop=True)
    df_val   = df_train_val.iloc[val_idx].reset_index(drop=True)
    return df_train, df_val, df_test

df_train, df_val, df_test = stratified_split(meta, label_col="label", test_size=0.15, val_size=0.15, seed=SEED)

def show_counts(df, name):
    print(f"\n{name} class counts:")
    print(df["label"].value_counts().reindex(class_names).fillna(0).astype(int).to_string())

print(f"✅ Split complete: Train={len(df_train)}, Val={len(df_val)}, Test={len(df_test)}")
show_counts(df_train, "Train")
show_counts(df_val,   "Val")
show_counts(df_test,  "Test")


✅ Split complete: Train=7009, Val=1503, Test=1503

Train class counts:
label
akiec     229
bcc       360
bkl       769
df         80
mel       779
nv       4693
vasc       99

Val class counts:
label
akiec      49
bcc        77
bkl       165
df         18
mel       167
nv       1006
vasc       21

Test class counts:
label
akiec      49
bcc        77
bkl       165
df         17
mel       167
nv       1006
vasc       22


In [19]:
print("df_train:", 'exists' if 'df_train' in globals() else 'MISSING')
print("df_val:",   'exists' if 'df_val'   in globals() else 'MISSING')
print("df_test:",  'exists' if 'df_test'  in globals() else 'MISSING')


df_train: exists
df_val: exists
df_test: exists


In [20]:
# --- STEP 5: Dataset, transforms, DataLoaders, and class weights ---

from PIL import Image
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import numpy as np
import torch

# Image size & batch size
IMG_SIZE = 224
BATCH    = 32

# Augmentations for train; deterministic eval transforms for val/test
train_tfms = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(8),
    transforms.ColorJitter(0.1, 0.1, 0.1, 0.05),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485,0.456,0.406], std=[0.229,0.224,0.225]),
])

eval_tfms = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485,0.456,0.406], std=[0.229,0.224,0.225]),
])

class SkinDataset(Dataset):
    def __init__(self, frame, tfm, cls_to_idx):
        self.frame = frame.reset_index(drop=True)
        self.tfm = tfm
        self.cls_to_idx = cls_to_idx

    def __len__(self):
        return len(self.frame)

    def __getitem__(self, i):
        row = self.frame.iloc[i]
        img = Image.open(row["image_path"]).convert("RGB")
        x   = self.tfm(img)
        y   = self.cls_to_idx[row["label"]]
        return x, y

# Build datasets
ds_train = SkinDataset(df_train, train_tfms, cls_to_idx)
ds_val   = SkinDataset(df_val,   eval_tfms,  cls_to_idx)
ds_test  = SkinDataset(df_test,  eval_tfms,  cls_to_idx)

# DataLoaders
loader_kwargs = dict(batch_size=BATCH, num_workers=2, pin_memory=(DEVICE=="cuda"))
train_loader = DataLoader(ds_train, shuffle=True,  **loader_kwargs)
val_loader   = DataLoader(ds_val,   shuffle=False, **loader_kwargs)
test_loader  = DataLoader(ds_test,  shuffle=False, **loader_kwargs)

# Class weights to handle imbalance (for CrossEntropyLoss)
counts = df_train["label"].value_counts().reindex(class_names).astype(int).values
class_weights = (counts.sum() / (len(counts) * counts)).astype(np.float32)
class_weights_t = torch.tensor(class_weights, device=DEVICE)

print("✅ DataLoaders ready.")
print("Train batches:", len(train_loader), "| Val batches:", len(val_loader), "| Test batches:", len(test_loader))
print("Class weights:", class_weights)


✅ DataLoaders ready.
Train batches: 220 | Val batches: 47 | Test batches: 47
Class weights: [ 4.3724265   2.7813492   1.302062   12.516071    1.2853475   0.21335728
 10.113997  ]


In [23]:
# --- STEP 6 (corrected): Train/eval loops, LR scheduler, checkpointing, reporting ---

import os, numpy as np
from tqdm.auto import tqdm
import torch
import torch.nn as nn
from torch.optim import Adam
from torch.optim.lr_scheduler import ReduceLROnPlateau
from sklearn.metrics import classification_report, confusion_matrix

# Where to save best model weights
SAVE_DIR = "/content/ckpts"
os.makedirs(SAVE_DIR, exist_ok=True)

def _current_lr(optimizer):
    for pg in optimizer.param_groups:
        return pg.get("lr", None)

def train_one_epoch(model, loader, criterion, optimizer):
    model.train()
    running_loss, correct, total = 0.0, 0, 0
    for x, y in tqdm(loader, leave=False):
        x, y = x.to(DEVICE), y.to(DEVICE)
        optimizer.zero_grad(set_to_none=True)
        logits = model(x)
        loss = criterion(logits, y)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * x.size(0)
        pred = logits.argmax(1)
        correct += (pred == y).sum().item()
        total += y.size(0)
    return running_loss / total, correct / total

@torch.no_grad()
def evaluate(model, loader, criterion):
    model.eval()
    running_loss, correct, total = 0.0, 0, 0
    all_preds, all_trues = [], []
    for x, y in loader:
        x, y = x.to(DEVICE), y.to(DEVICE)
        logits = model(x)
        loss = criterion(logits, y)

        running_loss += loss.item() * x.size(0)
        pred = logits.argmax(1)
        correct += (pred == y).sum().item()
        total += y.size(0)

        all_preds.extend(pred.cpu().numpy())
        all_trues.extend(y.cpu().numpy())
    return running_loss / total, correct / total, np.array(all_trues), np.array(all_preds)

def fit(model, train_loader, val_loader, *, epochs=20, lr=1e-3, save_name="model_best.pth"):
    """
    Trains model with class-weighted CrossEntropy, ReduceLROnPlateau on val acc,
    saves the best weights (by val acc) to SAVE_DIR/save_name, and returns (ckpt_path, best_val_acc).
    """
    ckpt_path = os.path.join(SAVE_DIR, save_name)
    criterion = nn.CrossEntropyLoss(weight=class_weights_t)
    optimizer = Adam(model.parameters(), lr=lr)
    # NOTE: some torch versions don't accept 'verbose' kwarg; keep it minimal-compatible
    scheduler = ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=2)

    best_val_acc = -1.0
    for ep in range(1, epochs + 1):
        tr_loss, tr_acc = train_one_epoch(model, train_loader, criterion, optimizer)
        val_loss, val_acc, y_true, y_pred = evaluate(model, val_loader, criterion)
        scheduler.step(val_acc)

        print(f"Epoch {ep:02d} | "
              f"train loss {tr_loss:.4f} acc {tr_acc:.4f} || "
              f"val loss {val_loss:.4f} acc {val_acc:.4f} || "
              f"lr { _current_lr(optimizer):.6f}")

        if val_acc > best_val_acc:
            best_val_acc = val_acc
            torch.save(model.state_dict(), ckpt_path)
            print(f"  ↳ saved best to {ckpt_path} (val acc {best_val_acc:.4f})")
    return ckpt_path, best_val_acc

def final_report(model, loader, name="Model"):
    """
    Prints accuracy, per-class precision/recall/F1, and confusion matrix on a loader.
    """
    criterion = nn.CrossEntropyLoss(weight=class_weights_t)
    loss, acc, y_true, y_pred = evaluate(model, loader, criterion)
    print(f"\n=== {name} ===")
    print(f"Accuracy: {acc:.4f}")
    print(classification_report(y_true, y_pred, target_names=class_names, digits=4))
    print("Confusion matrix:\n", confusion_matrix(y_true, y_pred))


In [24]:
# --- STEP 7: Baseline model — Plain CNN (3–4 conv blocks) ---

import torch.nn as nn
import torch.nn.functional as F
import torch

class PlainCNN(nn.Module):
    def __init__(self, num_classes=len(class_names)):
        super().__init__()
        self.conv1 = nn.Conv2d(3,   32, 3, padding=1)
        self.bn1   = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32,  64, 3, padding=1)
        self.bn2   = nn.BatchNorm2d(64)
        self.conv3 = nn.Conv2d(64, 128, 3, padding=1)
        self.bn3   = nn.BatchNorm2d(128)
        self.conv4 = nn.Conv2d(128, 256, 3, padding=1)
        self.bn4   = nn.BatchNorm2d(256)
        self.pool  = nn.MaxPool2d(2,2)
        self.drop  = nn.Dropout(0.5)
        # After 4 pools on 224x224 → 14x14 feature map
        self.fc1   = nn.Linear(256 * (IMG_SIZE//16) * (IMG_SIZE//16), 512)
        self.fc2   = nn.Linear(512, num_classes)

    def forward(self, x):
        x = self.pool(F.relu(self.bn1(self.conv1(x))))
        x = self.pool(F.relu(self.bn2(self.conv2(x))))
        x = self.pool(F.relu(self.bn3(self.conv3(x))))
        x = self.pool(F.relu(self.bn4(self.conv4(x))))
        x = x.flatten(1)
        x = self.drop(F.relu(self.fc1(x)))
        x = self.fc2(x)
        return x

plain = PlainCNN().to(DEVICE)

# Train with Step 6 utilities
plain_ckpt, best_val_acc_plain = fit(
    plain, train_loader, val_loader,
    epochs=20, lr=1e-3, save_name="plaincnn_best.pth"
)

# Load best weights and evaluate on test
plain.load_state_dict(torch.load(plain_ckpt, map_location=DEVICE))
final_report(plain, test_loader, "Plain CNN (Test)")


  0%|          | 0/220 [00:00<?, ?it/s]

Epoch 01 | train loss 5.9005 acc 0.2667 || val loss 1.7658 acc 0.5296 || lr 0.001000
  ↳ saved best to /content/ckpts/plaincnn_best.pth (val acc 0.5296)


  0%|          | 0/220 [00:00<?, ?it/s]

Epoch 02 | train loss 1.8403 acc 0.3052 || val loss 1.7760 acc 0.5828 || lr 0.001000
  ↳ saved best to /content/ckpts/plaincnn_best.pth (val acc 0.5828)


  0%|          | 0/220 [00:00<?, ?it/s]

...

Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7fe8627fce00>
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py", line 1664, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py", line 1647, in _shutdown_workers
    if w.is_alive():
       ^^^^^^^^^^^^
  File "/usr/lib/python3.12/multiprocessing/process.py", line 160, in is_alive
    assert self._parent_pid == os.getpid(), 'can only test a child process'
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
AssertionError: can only test a child process
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7fe8627fce00>
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py", line 1664, in __del__
Exception ignored in:     <function _MultiProcessingDataLoaderIter.__del__ at 0x7fe8627fce00>self._shutdown_workers()

Epoch 19 | train loss 1.1990 acc 0.5520 || val loss 1.1855 acc 0.6041 || lr 0.000031
  ↳ saved best to /content/ckpts/plaincnn_best.pth (val acc 0.6041)


  0%|          | 0/220 [00:00<?, ?it/s]

Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7fe8627fce00>
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py", line 1664, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py", line 1647, in _shutdown_workers
    if w.is_alive():
       ^^^^^^^^^^^^
  File "/usr/lib/python3.12/multiprocessing/process.py", line 160, in is_alive
    assert self._parent_pid == os.getpid(), 'can only test a child process'
Exception ignored in:  <function _MultiProcessingDataLoaderIter.__del__ at 0x7fe8627fce00>
Traceback (most recent call last):
   File "/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py", line 1664, in __del__
        self._shutdown_workers()    
  File "/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py", line 1647, in _shutdown_workers
    if w.is_alive(): 
^ ^  ^   ^ ^^^^^^^^^^^^^^^^
  Fi

Epoch 20 | train loss 1.2087 acc 0.5503 || val loss 1.1722 acc 0.6181 || lr 0.000031
  ↳ saved best to /content/ckpts/plaincnn_best.pth (val acc 0.6181)

=== Plain CNN (Test) ===
Accuracy: 0.6261
              precision    recall  f1-score   support

       akiec     0.3333    0.2653    0.2955        49
         bcc     0.3898    0.5974    0.4718        77
         bkl     0.3320    0.4909    0.3961       165
          df     0.1364    0.5294    0.2169        17
         mel     0.3255    0.4132    0.3641       167
          nv     0.9156    0.7008    0.7939      1006
        vasc     0.3333    0.8182    0.4737        22

    accuracy                         0.6261      1503
   macro avg     0.3951    0.5450    0.4303      1503
weighted avg     0.7227    0.6261    0.6585      1503

Confusion matrix:
 [[ 13  12   6  14   2   2   0]
 [  2  46  11   6   2   4   6]
 [  7  21  81  12  22  19   3]
 [  2   4   1   9   0   1   0]
 [  7   6  40   2  69  38   5]
 [  8  29 104  22 116 705  22]
 [

In [25]:
# --- STEP 8: Main model — ResNet50 (pretrained + fine-tune) ---

import torch
import torch.nn as nn
import torchvision.models as models

# Build pretrained ResNet50
resnet50 = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V2)
resnet50.fc = nn.Linear(resnet50.fc.in_features, len(class_names))
resnet50 = resnet50.to(DEVICE)

# Checkpoint name
RESNET_CKPT = os.path.join(SAVE_DIR, "resnet50_best.pth")

# ---- Warmup: train only the final classifier layer ----
for p in resnet50.parameters():
    p.requires_grad = False
for p in resnet50.fc.parameters():
    p.requires_grad = True

print("Warmup (fc only) ...")
_ckpt, _ = fit(
    resnet50, train_loader, val_loader,
    epochs=5, lr=1e-3, save_name="resnet50_best.pth"
)

# ---- Fine-tune: unfreeze last two stages (layer3 + layer4) + fc ----
for name, p in resnet50.named_parameters():
    p.requires_grad = ("layer3" in name) or ("layer4" in name) or ("fc" in name)

print("Fine-tuning ...")
_ckpt, _ = fit(
    resnet50, train_loader, val_loader,
    epochs=15, lr=1e-4, save_name="resnet50_best.pth"
)

# Load best weights and evaluate on test
resnet50.load_state_dict(torch.load(RESNET_CKPT, map_location=DEVICE))
final_report(resnet50, test_loader, "ResNet50 (Test)")


Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to /root/.cache/torch/hub/checkpoints/resnet50-11ad3fa6.pth


100%|██████████| 97.8M/97.8M [00:00<00:00, 166MB/s]


Warmup (fc only) ...


  0%|          | 0/220 [00:00<?, ?it/s]

Epoch 01 | train loss 1.5792 acc 0.5513 || val loss 1.3421 acc 0.6327 || lr 0.001000
  ↳ saved best to /content/ckpts/resnet50_best.pth (val acc 0.6327)


...

Epoch 13 | train loss 0.1003 acc 0.9406 || val loss 0.6988 acc 0.8629 || lr 0.000100
  ↳ saved best to /content/ckpts/resnet50_best.pth (val acc 0.8629)


  0%|          | 0/220 [00:00<?, ?it/s]

Epoch 14 | train loss 0.0973 acc 0.9462 || val loss 0.7367 acc 0.8676 || lr 0.000100
  ↳ saved best to /content/ckpts/resnet50_best.pth (val acc 0.8676)


  0%|          | 0/220 [00:00<?, ?it/s]

Epoch 15 | train loss 0.0867 acc 0.9512 || val loss 0.7033 acc 0.8629 || lr 0.000100

=== ResNet50 (Test) ===
Accuracy: 0.8683
              precision    recall  f1-score   support

       akiec     0.6333    0.7755    0.6972        49
         bcc     0.9014    0.8312    0.8649        77
         bkl     0.8960    0.6788    0.7724       165
          df     0.9167    0.6471    0.7586        17
         mel     0.5891    0.7126    0.6450       167
          nv     0.9299    0.9364    0.9331      1006
        vasc     0.9500    0.8636    0.9048        22

    accuracy                         0.8683      1503
   macro avg     0.8309    0.7779    0.7966      1503
weighted avg     0.8773    0.8683    0.8699      1503

Confusion matrix:
 [[ 38   0   2   0   6   3   0]
 [  3  64   2   0   4   4   0]
 [ 10   2 112   0  17  24   0]
 [  2   1   0  11   2   1   0]
 [  4   1   5   0 119  38   0]
 [  3   3   4   1  52 942   1]
 [  0   0   0   0   2   1  19]]
